In [5]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
from tqdm import tqdm
import utils
from dataset import StateActionEffectDataset
import environment
import copy

run_id = "0hcxpoi4"


In [6]:
run = wandb.init(entity="colorslab",project="multideepsym", resume="must", id=run_id)
run = wandb.Api().run("colorslab/multideepsym/" + run_id)
model = utils.create_model_from_config(run.config)
model.load("_best", from_wandb=True)

epoch,200
lr,0.0005
train_loss,1.15007
val_loss,1.55961


In [7]:
dataset = StateActionEffectDataset("v5", "")

In [8]:
count = 0
for e in dataset.effect:
    if torch.any(e[:,2].abs().max() > 0.1):
        count += 1
( count)/ len(dataset) * 100

40.87878787878788

In [12]:
k=108

In [13]:
import plotly.graph_objects as go
import plotly.express as px
sample = dataset[k]
s = copy.deepcopy(sample["state"]).numpy()
s = np.concatenate( [s,np.arange(0,13,1).reshape(-1,1)], axis = 1)
extensions = []
for obj in s:
    if obj[9] > 3:
        ext = copy.deepcopy(obj)
        ext2 = copy.deepcopy(obj)
        ds = 0.125
        
        ext[0] += ds * ext[7]
        ext[1] += ds * ext[8]
        ext2[0] -= ds * ext[7]
        ext2[1] -= ds * ext[8]
        extensions.append(ext)
        extensions.append(ext2)
extensions = np.array(extensions)
s = np.concatenate([s, np.array(extensions)], axis = 0)
fig = go.Figure()
fig = px.scatter_3d(s, x = 0, y = 1, z = 2, color = 10, hover_data=[10, 9] ,labels = {"0" : "x", "1": "y", "2":"z", "9":"type", "10": "id"}, opacity = 0.7)
fig.show()
# input()

In [11]:
s = copy.deepcopy(sample["action"]).numpy()
s = np.concatenate( [np.arange(0,13,1).reshape(-1,1),s], axis = 1)
s

array([[ 0.,  0.,  0.,  1., -1.,  0.,  1.,  0.,  1.],
       [ 1.,  0.,  0.,  1., -1.,  0.,  1.,  0.,  1.],
       [ 2.,  0.,  0.,  1., -1.,  0.,  1.,  0.,  1.],
       [ 3.,  0.,  0.,  1., -1.,  0.,  1.,  0.,  1.],
       [ 4.,  0.,  1.,  1., -1.,  0.,  1.,  0.,  1.],
       [ 5.,  1.,  0.,  1., -1.,  0.,  1.,  0.,  1.],
       [ 6.,  0.,  0.,  1., -1.,  0.,  1.,  0.,  1.],
       [ 7.,  0.,  0.,  1., -1.,  0.,  1.,  0.,  1.],
       [ 8.,  0.,  0.,  1., -1.,  0.,  1.,  0.,  1.],
       [ 9.,  0.,  0.,  1., -1.,  0.,  1.,  0.,  1.],
       [10.,  0.,  0.,  1., -1.,  0.,  1.,  0.,  1.],
       [11.,  0.,  0.,  1., -1.,  0.,  1.,  0.,  1.],
       [12.,  0.,  0.,  1., -1.,  0.,  1.,  0.,  1.]])

In [14]:
def single_iteration_data(data):
    for key in ["action", "state", "effect"]:
        data[key] = data[key].unsqueeze(0).expand(-1, 13, -1)
    data["pad_mask"] = data["pad_mask"].unsqueeze(0).expand(-1, 13)
    return data

In [16]:
with torch.no_grad():
    loss, pred = model.loss_with_pred(single_iteration_data(sample))
print(loss)

tensor(0.0405, device='cuda:0')


In [ ]:

fig = plt.figure(figsize=( 17, 39))
xy , yz , xz = [], [], []

state = sample["state"]
effect = sample["effect"]
x, y, z =state[:, 0].numpy() ,state[:, 1].numpy() ,state[:, 2].numpy() 
u,v,w, u2, v2,w2 = effect[:,0].numpy() ,effect[:,1].numpy() ,effect[:,2].numpy() ,pred[:,0].numpy() ,pred[:,1].numpy() ,pred[:,2].numpy() 

for i in range(13):
    xy.append(fig.add_subplot( 13,3,i*3 + 1 , xlabel="x", ylabel="y" ))
    yz.append(fig.add_subplot( 13,3,i*3 + 2 , xlabel="y", ylabel="z" ))
    xz.append(fig.add_subplot( 13,3,i*3 + 3  , xlabel="x", ylabel="z"))


for i in range(13):
    yz[i].plot([y[i],y[i] + v2[i]],[z[i], z[i]+ w2[i]],color ="r", label= "prediction", alpha= 0.7)
    xz[i].plot([x[i],x[i] + u2[i]],[z[i], z[i]+ w2[i]],color ="r", label= "prediction", alpha= 0.7)
    xy[i].plot([x[i],x[i] + u2[i]],[y[i], y[i]+ v2[i]],color ="r", label= "prediction", alpha= 0.7)
    xy[i].plot([x[i],x[i] + u[i]],[y[i], y[i]+ v[i]],  color ="b", label= "effect")
    yz[i].plot([y[i],y[i] + v[i]],[z[i], z[i]+ w[i]],  color ="b", label= "effect")
    xz[i].plot([x[i],x[i] + u[i]],[z[i], z[i]+ w[i]],  color ="b", label= "effect")
    if sample["pad_mask"][i] == 0:
        xy[i].set_ylabel("Non existent object")

xy[0].legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
                      ncols=2, mode="expand", borderaxespad=0.)
